In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score
import joblib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


df = pd.read_csv('/content/HeartDiseaseTrain-Test (3).csv')


drop_cols = ["fasting_blood_sugar", "rest_ecg"]
present_to_drop = [c for c in drop_cols if c in df.columns]

X = df.drop(["target"] + present_to_drop, axis=1)
y = df["target"].astype(int)


X = pd.get_dummies(X, drop_first=True)

#
feature_names = list(X.columns)
with open("features.json", "w") as f:
    json.dump(feature_names, f)
print(" Saved features.json")


RND = 42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RND
)


scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, "scaler.joblib")
print(" Saved scaler.joblib")


cw = class_weight.compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(cw))


def build_model(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation="relu", kernel_regularizer=keras.regularizers.l2(1e-3), input_shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(32, activation="relu", kernel_regularizer=keras.regularizers.l2(1e-3)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(16, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

model = build_model(X_train_scaled.shape[1])


early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=8, min_lr=1e-5, verbose=1)

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.15,
    epochs=100,
    batch_size=16,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


probs = model.predict(X_test_scaled).ravel()
preds = (probs >= 0.5).astype(int)
acc = accuracy_score(y_test, preds)
print(f"Test accuracy: {acc*100:.2f}%")


model.save("heart_model.keras")
print(" Saved heart_model.keras")

 Saved features.json
 Saved scaler.joblib
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5836 - loss: 0.8275 - val_accuracy: 0.7967 - val_loss: 0.6896 - learning_rate: 0.0010
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7564 - loss: 0.5822 - val_accuracy: 0.7724 - val_loss: 0.6314 - learning_rate: 0.0010
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7961 - loss: 0.5118 - val_accuracy: 0.7805 - val_loss: 0.5864 - learning_rate: 0.0010
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8099 - loss: 0.4612 - val_accuracy: 0.7724 - val_loss: 0.5534 - learning_rate: 0.0010
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8114 - loss: 0.4796 - val_accuracy: 0.7886 - val_loss: 0.5264 - learning_rate: 0.0010
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8354 - loss: 0.4641 - val_accuracy: 0.8130 - val_loss: 0.4891 - learning_rate: 0.0010
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8286 - loss: 0.4922 - val_accuracy: 0.82